In [1]:
import openai
import os
import random
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

# define global callback setting
from llama_index.core.settings import Settings
from llama_index.core.callbacks import CallbackManager

In [3]:

try:
    storage_context = StorageContext.from_defaults(
        persist_dir="storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [4]:
from llama_index.embeddings.openai import OpenAIEmbedding

if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["data/pdf/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["data/pdf/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"))
    uber_index = VectorStoreIndex.from_documents(uber_docs, embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"))

    # persist index
    lyft_index.storage_context.persist(persist_dir="storage/pdf/lyft")
    uber_index.storage_context.persist(persist_dir="storage/pdf/uber")

In [17]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)


hyde = HyDEQueryTransform(include_original=True)
lyft_hyde_query_engine = TransformQueryEngine(lyft_engine, hyde)
uber_hyde_query_engine = TransformQueryEngine(uber_engine, hyde)

In [10]:
print(lyft_hyde_query_engine.query("What is the revenue of Lyft?"))

The revenue of Lyft for the year ended December 31, 2021 was $3,208,323.


In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_hyde_query_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_hyde_query_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [13]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, verbose=True)

In [9]:
response = agent.query("How much exactly was the profit differences between Lyft and Uber in 2021?")
print(response)

> Running step 6ccd072b-bc97-4e54-91b9-2a549c207a43. Step input: How much exactly was the profit differences between Lyft and Uber in 2021?
Thought: The user is asking for the profit differences between Lyft and Uber in 2021. I need to use the financial tools to gather this information.
Action: lyft_10k
Action Input: {'input': 'What was the profit of Lyft in 2021?'}
Observation: Lyft's profit in 2021 was $1.0 billion.
> Running step 4aa92f46-81de-48af-bacd-1adba1c24ceb. Step input: None
Thought: I now have the profit information for Lyft in 2021. I need to gather the profit information for Uber in 2021 to calculate the profit difference between the two companies.
Action: uber_10k
Action Input: {'input': 'What was the profit of Uber in 2021?'}
Observation: Uber had a profit of $1.6 billion in 2021.
> Running step ef0b53d6-b413-4307-9976-fc2fe9b90540. Step input: None
Thought: I have the profit information for both Lyft and Uber in 2021. Now, I can calculate the profit difference between